# All the results and visualizations for the Sampta Paper

In [8]:
#Import everything
from Main.test_manifold_algorithms import test_manifold_algorithms as tma
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from Helpers.regression_helpers import *

## Checking if we have our data run

### Regression

In [9]:
reg_df = read_json_files_to_dataframe("/yunity/arusty/Graph-Manifold-Alignment/Results/RegressionData")
print(f"File Length: {len(reg_df)}")
reg_df.sample(5)

File Length: 1273


,method,csv_file,split,Percent_of_Anchors,Best_Params,CE,FOSCTTM,Random Forest OOB,Random Forest Emb,Nearest Neighbor,Nearest Neighbor (F1 score or RMSE),Random Forest (F1 score or RMSE),GRAE,Parameter STD
1256,MAGAN,AirfoilSelfNoise,turn,0.3,{'learning_rate': 0.005},"{'42': -0.691528928258315, '1738': -0.86598993...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.430520806904962, '1738': 0.4227645855...","{'42': 0.5359253745438446, '1738': 0.423060201...","{'42': 0.18383554475777042, '1738': 0.12670687...","{'42': 0.926908808340283, '1738': 0.9387776602...","{'42': 0.6989435189929266, '1738': 0.763041176...","{'42': {'RF-OOB': 0.3985966234426971, 'KNN': 0...",{}
474,RF-SPUD,OpticalNetwork,skewed,0.3,"{'knn': 2, 'OD_method': 'default', 'agg_method...",{'42': 0.011657090622319677},{'42': 0.00021164021164021165},{'42': 0.5861899370392037},{'42': 0.5084275472208392},{'42': 0.582752300787641},{'42': 0.6524469218652498},{'42': 0.7081772645747266},"{'42': {'RF-OOB': 0.09698112887229193, 'KNN': ...",{}
725,RF-SPUD,FacebookMetrics,skewed,0.3,"{'knn': 2, 'OD_method': 'default', 'agg_method...",{'42': 0.4025626334023852},{'42': 0.25324354657687986},{'42': 0.21602826126599206},{'42': 0.8039855213737158},{'42': 0.6847114734635191},{'42': 0.43940402057199623},{'42': 0.3464607321731235},"{'42': {'RF-OOB': 0.4233151430598803, 'KNN': 0...",{}
415,MALI,SML2010,turn,0.3,"{'knn': 2, 'mu': 0.1, 't': 30, 'transition_onl...","{'42': -0.2177191183458358, '1738': -0.2177191...","{'42': 0.0002639825287738327, '1738': 0.000263...","{'42': -0.03976198979914325, '1738': -0.041266...","{'42': -0.03158228462062329, '1738': -0.027769...","{'42': -0.009184588712009445, '1738': -0.00432...","{'42': 1.0002042737938708, '1738': 0.982097088...","{'42': 1.0112425584253204, '1738': 0.993491986...","{'42': {'RF-OOB': -0.040032598715101786, 'KNN'...",{}
1170,RF-NAMA,ConcreteCompressiveStrength,even,0.3,"{'OD_method': 'absolute_distance', 'agg_method...",{'42': -0.12092227306438774},{'42': 0.0012442266000565556},{'42': -0.09877785944557793},{'42': -0.13315317367423396},{'42': -0.02352732500193566},{'42': 0.9721240566466792},{'42': 1.022860198118425},"{'42': {'RF-OOB': -0.11211823247437147, 'KNN':...",{}


In [ ]:
#This helps me to see where my tests have failed
proc_reg_df = reg_df.drop(columns = ["Best_Params", "Parameter STD"])

#Average out the random seeds
for columns in ["CE", "FOSCTTM", "Random Forest OOB", "Random Forest Emb", "Nearest Neighbor", "Nearest Neighbor (F1 score or RMSE)", "Random Forest (F1 score or RMSE)"]:
    proc_reg_df[columns] = [np.mean(list(d.values())) for d in proc_reg_df[columns].values]

proc_reg_df["Combined_Metric"] = proc_reg_df["CE"] - proc_reg_df["FOSCTTM"]

baselines = pd.read_csv("/yunity/arusty/Graph-Manifold-Alignment/Results/RegressionData/BaselineDataTrainTest.csv", index_col=0).rename(columns = {"KNN_BL_A": "A_Classification_Score", "KNN_BL_B": "B_Classification_Score"})
baselines_grouped = baselines.groupby(by = ["csv_file", "split"]).max().drop(columns = ["KNN"]).reset_index()

# Group by csv_file and select the row with the 3rd smallest knn value
third_smallest_knn = baselines.groupby(by = ["csv_file", "split"]).apply(lambda x: x.nsmallest(3, 'KNN').iloc[-1]).reset_index(drop=True)

# Ensure the resulting DataFrame has the same format as baselines_grouped
third_smallest_knn_grouped = third_smallest_knn.groupby(by=["csv_file", "split"]).agg({
    "A_Classification_Score": "max",
    "B_Classification_Score": "max"
}).reset_index()
proc_reg_df = proc_reg_df.merge(third_smallest_knn_grouped, how = "left")


reg_results_df = plt_methods_by_CSV_max(df = proc_reg_df, metric = "Combined_Metric", return_df=True)#.head(13) #, split = "skewed"
reg_results_df

,csv_file,SSMA,MAGAN,DTA,SPUD,MASH,MASH-,RF-MASH-,NAMA,RF-NAMA,PCR,JLMA,MASH_RF,MALI_RF,MALI,SPUD_RF,BL_A,BL_B
0,ConcreteCompressiveStrength,0.579742,-0.171202,0.900178,0.355952,0.640715,0.769619,0.773025,-0.159360,-0.122166,0.900479,0.845669,0.628309,-0.355010,-0.128310,-0.074225,0.610397,0.715961
1,Hydrodynamics,0.664402,0.532023,0.899806,0.799632,0.671012,0.706608,0.878167,-0.054671,-0.077427,0.895496,0.796840,0.720924,-0.211597,0.759538,0.932535,0.585006,0.790342
2,FacebookMetrics,0.592234,-0.007211,0.759274,0.262851,0.684521,0.685547,0.592401,0.320081,-0.121600,0.744296,0.660645,0.690289,-0.114347,0.503568,0.543361,0.778232,0.691495
3,CommunityCrime,0.497916,-0.024250,0.793918,0.535569,0.710046,0.712440,0.583943,-0.142984,-0.192051,0.786104,0.740419,0.642806,-0.290245,-0.139376,0.252467,0.592101,0.599702
4,OpticalNetwork,0.473332,-0.147061,0.908100,0.518772,0.747207,0.676237,0.924531,-0.180715,-0.182083,0.910902,0.854887,0.774292,-0.350329,0.451797,0.022364,0.615307,0.798834
5,Automobile,0.757670,0.256283,0.882649,0.715318,0.767879,0.821432,0.841581,0.343892,0.098240,0.881048,0.804208,0.723531,-0.139808,0.795496,0.769123,0.624074,0.717398
6,ConcreteSlumpTest,0.513335,0.395941,0.890893,0.623485,0.780732,0.751162,0.570243,0.123935,0.102971,0.816872,0.875523,0.318204,-0.171756,0.852770,0.793678,0.745238,0.779765
7,ComputerHardware,0.962834,0.096644,0.964519,0.686117,0.799729,0.864906,0.581483,0.045628,0.069540,0.949490,0.845673,0.809029,-0.189637,0.927786,0.641705,0.656128,0.626540
8,IstanbulStock,0.674731,0.292751,0.844776,0.441662,0.805451,0.811195,0.714202,0.195857,-0.124704,0.834900,0.730128,0.642801,-0.316266,0.366207,0.766318,0.714484,0.771851
9,AirfoilSelfNoise,0.488030,-0.113552,0.962474,0.531724,0.873111,0.859316,0.797466,-0.135461,-0.169613,0.924405,0.848466,0.690631,-0.270939,-0.182528,-0.057775,0.618199,0.618291


### Classification

In [11]:
class_df = read_json_files_to_dataframe("/yunity/arusty/Graph-Manifold-Alignment/Results/ManifoldData")
print(f"File Length: {len(class_df)}")
class_df.sample(5)

File Length: 1524


,method,csv_file,split,Percent_of_Anchors,Best_Params,CE,FOSCTTM,Random Forest OOB,Random Forest Emb,Nearest Neighbor,Nearest Neighbor (F1 score or RMSE),Random Forest (F1 score or RMSE),GRAE,Parameter STD
836,RF-SPUD,hepatitis,turn,0.3,"{'knn': 29, 'OD_method': 'default', 'agg_metho...","{'42': 0.6739130434782609, '1738': 0.594202898...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.75, '1738': 0.7227272727272728, '5271...","{'42': 0.5892857142857143, '1738': 0.785714285...","{'42': 0.625, '1738': 0.75, '5271': 0.71428571...","{'42': 0.632733224222586, '1738': 0.7490322580...","{'42': 0.5977554360533084, '1738': 0.784884792...","{'42': {'RF-OOB': 0.7363636363636363, 'KNN': 0...",{}
741,RF-MASH-,water_potability,distort,0.3,"{'knn': 2, 'page_rank': 'full', 'DTM': 'log', ...","{'42': 0.006464445549477871, '1738': 0.0054699...","{'42': 0.44171910286561655, '1738': 0.44487133...","{'42': 0.29384328358208955, '1738': 0.31560945...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': {'RF-OOB': 0.029850746268656716, 'KNN':...","{'knn': 0.003551316105727078, 'page_rank': 0.0..."
154,SPUD,ionosphere,random,0.3,"{'knn': 26, 'OD_method': 'default', 'agg_metho...","{'42': 0.6581196581196581, '1738': 0.549857549...","{'42': 0.00018668679637340608, '1738': 7.30513...","{'42': 0.8892857142857142, '1738': 0.875, '527...","{'42': 0.8802816901408451, '1738': 0.957746478...","{'42': 0.7887323943661971, '1738': 0.802816901...","{'42': 0.7748898891631576, '1738': 0.781966565...","{'42': 0.8791754699483665, '1738': 0.957217329...","{'42': {'RF-OOB': 0.8732142857142857, 'KNN': 0...","{'knn': 0.09832093240184661, 'OD_method': 0.00..."
271,MASH-,tic-tac-toe,even,0.3,"{'knn': 2, 'page_rank': 'full', 'DTM': 'log', ...",{'42': 0.627348643006263},{'42': 0.16627259295417995},{'42': 0.783289817232376},{'42': 0.75},{'42': 0.6510416666666666},{'42': 0.5790531346317792},{'42': 0.7434301088724816},"{'42': {'RF-OOB': 0.7767624020887729, 'KNN': 0...","{'knn': 0.06494346720783084, 'page_rank': 0.01..."
1342,NAMA,iris,even,0.3,"{'OD_method': 'absolute_distance', 'agg_method...",{'42': 0.5133333333333333},{'42': 0.002444444444444445},{'42': 0.9333333333333333},{'42': 0.95},{'42': 0.9666666666666667},{'42': 0.9666666666666667},{'42': 0.9498412698412698},"{'42': {'RF-OOB': 0.9041666666666667, 'KNN': 0...","{'OD_method': 0.03999999999999998, 'agg_method..."


In [18]:
#This helps me to see where my tests have failed
proc_class_df = class_df.drop(columns = ["Best_Params", "Parameter STD"])

#Average out the random seeds
for columns in ["CE", "FOSCTTM", "Random Forest OOB", "Random Forest Emb", "Nearest Neighbor", "Nearest Neighbor (F1 score or RMSE)", "Random Forest (F1 score or RMSE)"]:
    proc_class_df[columns] = [np.mean(list(d.values())) for d in proc_class_df[columns].values]

proc_class_df["Combined_Metric"] = proc_class_df["CE"] - proc_class_df["FOSCTTM"]

baselines = pd.read_csv("/yunity/arusty/Graph-Manifold-Alignment/Results/ManifoldData/Data_DataFrame.csv", keep_default_na=False, na_values=['', 'NaN'], index_col= None
                          )[['csv_file', 'split', 'KNN', 'A_Classification_Score',
                            'B_Classification_Score']]

baselines_grouped = baselines.groupby(by = ["csv_file", "split"]).max().drop(columns = ["KNN"]).reset_index()

# Group by csv_file and select the row with the 3rd smallest knn value
third_smallest_knn = baselines.groupby(by = ["csv_file", "split"]).apply(lambda x: x.nsmallest(3, 'KNN').iloc[-1]).reset_index(drop=True)

# Ensure the resulting DataFrame has the same format as baselines_grouped
third_smallest_knn_grouped = third_smallest_knn.groupby(by=["csv_file", "split"]).agg({
    "A_Classification_Score": "max",
    "B_Classification_Score": "max"
}).reset_index()
proc_class_df = proc_class_df.merge(third_smallest_knn_grouped, how = "left")


cls_results_df = plt_methods_by_CSV_max(df = proc_class_df, metric = "Combined_Metric", return_df=True)#.head(13) #, split = "skewed"
cls_results_df

,csv_file,SSMA,MAGAN,DTA,SPUD,MASH,MASH-,RF-MASH-,NAMA,RF-NAMA,PCR,JLMA,MASH_RF,MALI_RF,MALI,SPUD_RF,BL_A,BL_B
0,zoo,NaN,NaN,NaN,0.207999,0.182413,0.187864,0.022380,0.158122,0.098559,NaN,NaN,0.222214,NaN,NaN,0.221782,0.415842,0.366337
1,water_potability,NaN,NaN,NaN,0.128309,0.249229,0.246931,-0.310442,0.002486,0.016896,NaN,NaN,0.252443,NaN,NaN,0.016111,0.294878,0.260567
2,winequality-red,NaN,NaN,NaN,0.106834,0.454108,0.447723,0.047582,0.026507,0.046990,NaN,NaN,0.370604,NaN,NaN,0.070614,0.315197,0.273296
3,b,NaN,NaN,NaN,1.000000,0.623400,0.454989,0.483450,0.870000,0.686667,NaN,NaN,0.558706,NaN,NaN,0.863333,NaN,NaN
4,hill_valley,NaN,NaN,NaN,0.546205,0.696134,0.691128,0.728979,0.570952,0.539601,NaN,NaN,0.696134,NaN,NaN,0.684565,0.790429,0.790429
5,heart_disease,NaN,NaN,NaN,0.626263,0.717172,0.717172,0.558401,0.515140,0.606049,NaN,NaN,0.701010,NaN,NaN,0.666653,0.757576,0.757576
6,Medicaldataset,NaN,NaN,NaN,0.613683,0.752287,0.755764,0.657278,0.561031,0.806822,NaN,NaN,0.750689,NaN,NaN,0.960425,0.717968,0.674754
7,audiology,NaN,NaN,NaN,0.714571,0.775654,0.753457,0.716939,0.504321,0.326316,NaN,NaN,0.755416,NaN,NaN,0.774715,0.900000,0.889474
8,heart_failure,NaN,NaN,NaN,0.749164,0.779698,0.770797,0.714860,0.658863,0.750502,NaN,NaN,0.736455,NaN,NaN,0.891639,0.799331,0.799331
9,hepatitis,NaN,NaN,NaN,0.760870,0.780923,0.789388,0.632950,0.695652,0.568084,NaN,NaN,0.744933,NaN,NaN,0.808696,0.797101,0.789855
